Perhaps mark each pdf with a code denoting the bank type, so this file only has to do a simple string parse to know what to do. i.e. 'JohnDoeCAP.pdf', 'MarkAnthonyNED.pdf', etc. 

Sometimes Description is long and overflows. I cut it off, but should we keep it all?

Formulae:
Number of transactions per month/amount of customers
Total money in/out subtract out-in for net
Bin description by types/prevalence 

In [21]:
import math 
import tabula as tb
from tabula import read_pdf

#constant conversion factor, per adobe
#pdf measurer pdf unit = 1/72 in
cf =72  

#all measurements in format [top,left,bottom,width]

#page counter
from PyPDF2 import PdfFileReader
def count_pdf_pages(file):
    with open(file, 'rb') as pdf:
        page_count = PdfFileReader(pdf).getNumPages()
    return page_count

In [22]:
import pandas as pd
#CAPITEC
#complete

def CAP_scan(filename):
    #capitec measurements
    #first page:
    page_1_box = [3.77, 0.4, 9.8, 8.5,]
    #middle pages:
    middle_pages_box = [0.3, 0.4, 10.2, 8.5]

  
    page_count = count_pdf_pages(filename)

    #convert boxes
    for i in range(0, len(page_1_box)):
        page_1_box[i] *= cf

    for i in range(0, len(middle_pages_box)):
        middle_pages_box[i] *= cf

    #get first page
    page_1_table = tb.read_pdf(filename, pages=1,area=[page_1_box],output_format="dataframe", stream=True)
    page_1_df = page_1_table[0].drop(labels='Posting Date', axis=1)

    #clean first page
    page_1_df = page_1_df.rename(columns={'Transaction': 'Transaction Date'})
    page_1_df = page_1_df.drop(index=0)
    page_1_df = page_1_df.dropna(subset=['Transaction Date','Balance (R)'])

    #append first page
    total_df = pd.DataFrame(columns=page_1_df.columns)
    total_df = pd.concat([total_df, page_1_df])


    for page in range(2, page_count+1):

        #get middle page
        middle_pages_table = tb.read_pdf(filename, pages=page, area=[middle_pages_box],
                                     output_format="dataframe", stream=True, pandas_options={'header': None})
        middle_df = middle_pages_table[0]

        #clean middle page
        middle_df = middle_df.dropna(axis='columns', how='all')
        middle_df.columns = page_1_df.columns


        middle_df = middle_df.dropna(subset=['Transaction Date','Balance (R)'])
        middle_df['Transaction Date'] = middle_df['Transaction Date'].map(lambda x: str(x).split()[-1])

        #append to total df
        total_df = pd.concat([total_df, middle_df])

        
    #combine money in and money out columns
    total_df = total_df.bfill(axis=1)
    
    total_df = total_df.drop(['Money Out (R)'], axis=1)
    
    total_df = total_df.rename(columns={'Money In (R)': 'Amount'})
    total_df = total_df.rename(columns={'Balance (R)': 'Balance'})

    #change dates to datetime format
    total_df['Transaction Date'] = pd.to_datetime(total_df['Transaction Date'])
    
    return total_df




In [23]:
import pandas as pd
#Mercantile Bank
#can these be longer than the one provided??

    
def MER_scan(filename):

    #mercantile measurements
    #first page:
    page_1_box = [4.2, 0.4, 8.9, 7.7,]
    #middle pages:
    middle_pages_box = [2.4, 0.4, 10.1, 7.7]

    page_count = count_pdf_pages(filename)

    #convert boxes
    for i in range(0, len(page_1_box)):
        page_1_box[i] *= cf

    for i in range(0, len(middle_pages_box)):
        middle_pages_box[i] *= cf

    #get first page
    page_1_table = tb.read_pdf(filename, pages=1,area=[page_1_box],output_format="dataframe", stream=True)
    page_1_df = page_1_table[0].drop(labels='Post', axis=1)

    #page_1_df
    #clean first page
    page_1_df = page_1_df.rename(columns={'Trans.': 'Transaction Date'})
    page_1_df = page_1_df.drop(index=0)
    page_1_df = page_1_df.dropna(subset=['Transaction Date','Balance'])

    #append first page
    total_df = pd.DataFrame(columns=page_1_df.columns)
    total_df = pd.concat([total_df, page_1_df])


    for page in range(2, page_count+1):

        #get middle page
        middle_pages_table = tb.read_pdf(filename, pages=page, area=[middle_pages_box],
                                     output_format="dataframe", stream=True, pandas_options={'header': None})
        middle_df = middle_pages_table[0]

        #clean middle page
        middle_df = middle_df.drop(labels=0, axis=1)
        middle_df = middle_df.drop(index=[0,1])
        middle_df = middle_df.dropna(axis='columns', how='all')
        middle_df.columns = page_1_df.columns
        middle_df = middle_df.dropna(subset=['Transaction Date','Balance'])

        #append to total df
        total_df = pd.concat([total_df, middle_df])

        
    #drop reference
    total_df = total_df.drop(['Reference'], axis=1)
    
    #turn nan charges into 0 
    total_df['Fees'] = total_df['Fees'].fillna(0)
    total_df['Amount'] = total_df['Amount'].fillna(0)
    total_df['Balance'] = total_df['Balance'].fillna(0)


    
    #numerize fees, amount, and balance
  

    total_df["Fees"] = total_df["Fees"].replace(' ', '', regex=True).astype(float)
    total_df["Amount"] = total_df["Amount"].replace(' ', '', regex=True).astype(float)
    total_df["Balance"] = total_df["Balance"].replace(' ', '', regex=True).astype(float)


    #combine fees and amount and drop fees
    total_df["Amount"] = total_df["Fees"] + total_df["Amount"]
    total_df = total_df.drop(["Fees"], axis=1)

    #change dates to datetime format
    total_df['Transaction Date'] = pd.to_datetime(total_df['Transaction Date'])
    
    return total_df


In [24]:
import pandas as pd
#Nedbank

    
def NED_scan(filename):

    #mercantile measurements
    #first page, first box:
    page_1_box_a = [4.2, 0.4, 6, 7.7]
    #first page, second box:
    page_1_box_b = [5.8, 0.4, 8.2, 7.7]



    #MUST BE ADJUSTED IF NARRATIVE DESCRIPTION HAS !6 ELEMENTS
    #second page
    page_2_box = [3.1, 0.4, 10.5, 7.9]

    #middle pages
    middle_pages_box = [1.6, 0.4, 10.5, 7.9]

    filename = '#NED#Nedbank.pdf'
    page_count = count_pdf_pages(filename)

    #convert boxes
    for i in range(0, len(page_1_box_a)):
        page_1_box_a[i] *= cf

    for i in range(0, len(page_1_box_b)):
        page_1_box_b[i] *= cf

    for i in range(0, len(page_2_box)):
        page_2_box[i] *= cf

    for i in range(0, len(middle_pages_box)):
        middle_pages_box[i] *= cf

    #get first page analytics
    page_1_table_a = tb.read_pdf(filename, pages=1,area=[page_1_box_a],output_format="dataframe", stream=True)

    page_1_table_b = tb.read_pdf(filename, pages=1,area=[page_1_box_b],output_format="dataframe", stream=True)



    #second page
    #get second page
    page_2_table = tb.read_pdf(filename, pages=2,area=[page_2_box],output_format="dataframe", stream=True)
    page_2_df = page_2_table[0]#.drop(labels='Tran list no', axis=1)


    #if col not equal date edge case
    #clean second page
    page_2_df = page_2_df.rename(columns={'Date': 'Transaction Date', 'Balance (R)': 'Balance', 'Fees (R)': 'Fees', 'Debits (R)': 'Debits', 
                                          'Credits (R)': 'Credits'})
    page_2_df = page_2_df.dropna(subset=['Transaction Date','Balance'])

    #append second page
    total_df = pd.DataFrame(columns=page_2_df.columns)
    total_df = pd.concat([total_df, page_2_df])


    for page in range(3, page_count+1):

        #get middle page
        middle_pages_table = tb.read_pdf(filename, pages=page, area=[middle_pages_box],
                                     output_format="dataframe", stream=True, pandas_options={'header': None})
        middle_df = middle_pages_table[0]
        middle_df.columns = middle_df.iloc[0]
        middle_df = middle_df.drop(0)

        #edge case when tran list no and date merge
        if middle_df.columns[1] != 'Date':
            middle_df[middle_df.columns[1]] = middle_df[middle_df.columns[0]]

        middle_df.columns = page_2_df.columns

        #append to total df
        total_df = pd.concat([total_df, middle_df])



    total_df = total_df.dropna(subset=['Transaction Date','Balance'])
    total_df = total_df.drop(labels='Tran list no', axis=1)



    #turn debits and credits into amount
    #turn nan charges into 0 
    total_df['Fees'] = total_df['Fees'].fillna(0)
    total_df['Debits'] = total_df['Debits'].fillna(0)
    total_df['Credits'] = total_df['Credits'].fillna(0)
    total_df['Balance'] = total_df['Balance'].fillna(0)


    #numerize fees, amount, and balance
    total_df["Fees"] = total_df["Fees"].replace(' ', '', regex=True).replace(',', '', regex=True).astype(float)
    total_df["Debits"] = total_df["Debits"].replace(' ', '', regex=True).replace(',', '', regex=True).replace('\*', '', regex=True).astype(float)
    total_df["Credits"] = total_df["Credits"].replace(' ', '', regex=True).replace(',', '', regex=True).astype(float)
    total_df["Balance"] = total_df["Balance"].replace(' ', '', regex=True).replace(',', '', regex=True).astype(float)

    #make debits and fees negative
    total_df["Debits"] = total_df["Debits"] *-1
    total_df["Fees"] = total_df["Fees"] *-1

    #combine fees, credits and debits into credits, rename as amount, drop fees and debits
    total_df["Credits"] = total_df["Fees"] + total_df["Credits"] + total_df["Debits"]
    total_df = total_df.drop(["Fees", "Debits"], axis=1)
    total_df = total_df.rename(columns={'Credits': 'Amount'})

    #change dates to datetime format
    total_df['Transaction Date'] = pd.to_datetime(total_df['Transaction Date'])

    return total_df

In [25]:
#run scanner

filename = input("enter name of file to be scanned\n")
if filename.startswith("#CAP#"):
    df = CAP_scan(filename)
elif filename.startswith("#FNB#"):
    df = FNB_scan(filename)
elif filename.startswith("#NED#"):
    df = NED_scan(filename)
elif filename.startswith("#MER#"):
    df = MER_scan(filename)


enter name of file to be scanned
#CAP#Capitec.pdf


In [26]:
#generate report
final_bal = df['Balance'].iat[-1]
